In [2]:
import os
import pickle
import numpy as np
from PIL import Image
from collections import OrderedDict
import matplotlib
import matplotlib.pyplot as plt

In [3]:
model_path = "C:\\Users\\AndrewHuang\\Documents\\GitHub\\02958343\\02958343\\1a0bc9ab92c915167ae33d942430658c\\model.obj"

In [4]:
vertex_list = []
face_list = []

In [5]:
def get_vertex_face(model_path):
    vertex_list = []
    face_list = []
    with open(model_path, "r") as f:
        for line in f:
            if line.startswith("#") or line.startswith("mtllib") or line.startswith("g") or line.startswith("usemtl"):
                continue
            elif line.startswith("v "):
                #print(line.split())
                x = float(line.split(" ")[1])
                y = float(line.split(" ")[2])
                z = float(line.split(" ")[3])
                vertex_list.append([x, y, z])
                continue
            elif line.startswith("f "):
                v1 = int(line.split(" ")[1].split("/")[0])
                v2 = int(line.split(" ")[2].split("/")[0])
                v3 = int(line.split(" ")[3].split("/")[0])
                face_list.append([v1, v2, v3])
                continue
    return vertex_list, face_list

In [21]:
### done, successfully reconstruct a 3d model without texture
with open("./test_ply.ply", "w") as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex " + str(len(vertex_list)) + "\n")
    f.write("property float x\n")
    f.write("property float y\n")
    f.write("property float z\n")
    f.write("element face " + str(len(face_list)) + "\n")
    f.write("property list uint8 uint32 vertex_indices\n")
    f.write("end_header\n")
    for vertex in vertex_list:
        f.write(str(vertex[0]) + " " + str(vertex[1]) + " " + str(vertex[2]) + "\n")
    for face in face_list:
        f.write(str(3) + " " + str(face[0] - 1) + " " + str(face[1] - 1) + " " + str(face[2] - 1) + "\n")

In [8]:
model_list = []
for file in os.listdir("./rotate"):
    if file.startswith("model_name_"):
        print(os.path.join("./rotate", file))
        with open(os.path.join("./rotate", file), "rb") as f:
            model_list_partial = pickle.load(f)
        for model in model_list_partial:
            model_list.append(model.split("/")[-2])
    #break

precision_constant = 10000
    
model_path = "C:\\Users\\AndrewHuang\\Documents\\GitHub\\02958343\\02958343"
for idx, model in enumerate(model_list):
    #os.system("blender --background --python 3dloc_2dcoor.py -- --views 18 " + os.path.join(model_path, model, "model.obj"))
    with open("./vertex2pixel", "rb") as f:
        vertex2pixel = pickle.load(f)
    with open("./pixel2vertex", "rb") as f:
        pixel2vertex = pickle.load(f)
    vertex2color = OrderedDict()
    for az in range(18):
        for el in range(3):
            real_img_path = "C:\\tmp\\test\\" + model + "\\" + model + "_az_" + str(az * 20) + "_el_" + str(el * 10) + "_albedo.png0001.png"
            realimg = np.array(Image.open(real_img_path).convert("RGB"))
            syn_img_path = "./rotate/rotate/" + str(idx * 1000 + az * 10 + el) + ".0.jpg"
            synimg = np.array(Image.open(syn_img_path))
            '''
            for vertex_pixel_pair in vertex2pixel["az_" + str(az * 20) + "_el_" + str(el * 10)]:
                if not repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int)) in vertex2color:
                    vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))] = []
                vertex2color[repr((precision_constant * np.array(vertex_pixel_pair[:3])).astype(int))].append(realimg[vertex_pixel_pair[3], vertex_pixel_pair[3]])
            '''
            for i in range(128):
                for j in range(128):
                    if pixel2vertex[az, el, i, j, 0] != 1234567.0:
                        if not repr((10000 * pixel2vertex[az, el, i, j, 1:]).astype(int)) in vertex2color:
                            vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))] = []
                        #vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append(realimg[i, j])
                        vertex2color[repr((precision_constant * pixel2vertex[az, el, i, j, 1:]).astype(int))].append(synimg[i, j])
            
    vertex_list, face_list = get_vertex_face(os.path.join(model_path, model, "model.obj"))
    color_list = np.zeros(shape=(len(vertex_list), 3)).astype(int)
    for idx, vertex in enumerate(vertex_list):
        color = [0, 0, 0]
        if repr((np.array(vertex) * precision_constant).astype(int)) in vertex2color:
            color = np.mean(vertex2color[repr((np.array(vertex) * precision_constant).astype(int))], axis=0).astype(int)
        color_list[idx] = color
    vertex2neighbor = np.zeros(shape=(len(vertex_list), 2)) # idx, dist
    vertex2neighbor[:, 1] = 2.0
    for face in face_list:
        v1, v2, v3 = face[0] - 1, face[1] - 1, face[2] - 1;
        d12 = np.linalg.norm(np.array(vertex_list[v1]) - np.array(vertex_list[v2]))
        d23 = np.linalg.norm(np.array(vertex_list[v2]) - np.array(vertex_list[v3]))
        d31 = np.linalg.norm(np.array(vertex_list[v3]) - np.array(vertex_list[v1]))
        if d12 < vertex2neighbor[v1, 1]:
            vertex2neighbor[v1, 0] = v2
            vertex2neighbor[v1, 1] = d12
        if d12 < vertex2neighbor[v2, 1]:
            vertex2neighbor[v2, 0] = v1
            vertex2neighbor[v2, 1] = d12
        if d23 < vertex2neighbor[v2, 1]:
            vertex2neighbor[v2, 0] = v3
            vertex2neighbor[v2, 1] = d23
        if d23 < vertex2neighbor[v3, 1]:
            vertex2neighbor[v3, 0] = v2
            vertex2neighbor[v3, 1] = d23
        if d31 < vertex2neighbor[v1, 1]:
            vertex2neighbor[v1, 0] = v3
            vertex2neighbor[v1, 1] = d31
        if d31 < vertex2neighbor[v3, 1]:
            vertex2neighbor[v3, 0] = v1
            vertex2neighbor[v3, 1] = d31
    cnt_loop = 0
    while True:
        cnt_loop += 1
        all_vertex_not_colored = True
        for idx, vertex in enumerate(vertex_list):
            if np.sum(color_list[idx]) == 0 and np.sum(color_list[int(vertex2neighbor[idx, 0])]) != 0:
                color_list[idx] = color_list[int(vertex2neighbor[idx, 0])]
                all_vertex_not_colored = False
        if all_vertex_not_colored:
            break
        if cnt_loop >= 1000:
            break
            

    with open("./test_ply_syn_mindist_propagation.ply", "w") as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write("element vertex " + str(len(vertex_list)) + "\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("property uchar red\n")
        f.write("property uchar green\n")
        f.write("property uchar blue\n")
        f.write("element face " + str(len(face_list)) + "\n")
        f.write("property list uint8 uint32 vertex_indices\n")
        f.write("end_header\n")
        for idx, vertex in enumerate(vertex_list):
            color = color_list[idx]
            f.write(str(vertex[0]) + " " + str(vertex[1]) + " " + str(vertex[2]) + " " + str(color[0]) + " " + str(color[1]) + " " + str(color[2]) + "\n")
        for face in face_list:
            f.write(str(3) + " " + str(face[0] - 1) + " " + str(face[1] - 1) + " " + str(face[2] - 1) + "\n")
    if idx >= 0:
        break

./rotate\model_name_0
./rotate\model_name_1000
./rotate\model_name_2000
./rotate\model_name_3000
./rotate\model_name_5000
./rotate\model_name_6000
./rotate\model_name_7000
./rotate\model_name_8000
./rotate\model_name_9000


In [9]:
cnt = 0
for color in color_list:
    if np.sum(color) == 0.0:
        cnt += 1
print(cnt)

1776


In [27]:
model_list[0]

'7b2dcc7c4c879e9116f7ad0d1093bd39'

In [ ]:
# test_ply_real: threshold = 1.0, with min dist, real pixel, const=10000
# test_plt_syn: threshold = 1.0, with min dist, syn pixel, const=10000
# with min dist:threshold = 1.0 or 0.2 -> blurry, but why? 
#    - instable ray_cast?
# threshold = 0.2: more vertices left blank, with const=1e6
#    - fixed with const=10000, see test_ply_real_threshold20
# 0615: epoch=3, lossGB about 2.0, run with real depth image